In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import gsw
import xarray as xr
from scipy.interpolate import RegularGridInterpolator as rgi
import pickle

### Overview
This code is used to interpolate variables onto the particle trajectories. The output of this processing step is saved and used in subsequent analysis.

#### Load all particles into a dictionary

In [ ]:
# the folder 'particle_output' can be downloaded from doi:10.6084/m9.figshare.13322435
dfdict = {}
for d in np.arange(44,63):
    conn = sqlite3.connect("particle_output/ini_day"+str(d)+"_5m_forward.db")
    cur = conn.cursor()
    df = pd.read_sql_query("select DOY, ID, x, y, z, u, v, w, density, vorticity, PV from particles;", conn)
    df1 = df.pivot(index='DOY', columns='ID')
    dfdict['df'+str(d)] = df1

#### Compute model gradients and interpolate onto particles

In [5]:
for d0 in np.arange(44,63):
    df1 = dfdict['df'+str(d0)]
    df1div = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Q1 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Q2 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1nonlin = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1ag = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1wz = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1rx = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1ry = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1rz = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1ml = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1vor2 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1vor3 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    dfML = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    for d in df1.index.values:
        day0 = int(np.floor(d*86400/108/100)*100)
        day1 = day0+100
        dd = np.mod(int(d*86400/108),100)
        filename = './../../exp_IRENE_1wiggle_cooled_bottom_friction/full_'+str(day0)+'.cdf'
        DS0 = xr.open_dataset(filename)
        DS0 = DS0.assign_coords(x = DS0.xc*1000,y = DS0.yc*1000,sigma = DS0.zc[:,1,1])
        filename = './../../exp_IRENE_1wiggle_cooled_bottom_friction/full_'+str(day1)+'.cdf'
        DS1 = xr.open_dataset(filename)
        DS1 = DS1.assign_coords(x = DS1.xc*1000,y = DS1.yc*1000,sigma = DS0.zc[:,1,1])
        DS = DS0*(1-dd/100)+DS1*dd/100

        rx = DS.rho.differentiate('x')
        ry = DS.rho.differentiate('y')
        rz = DS.rho.differentiate('sigma')
        wx = DS.w.differentiate('x')
        wy = DS.w.differentiate('y')
        wz = DS.w.differentiate('sigma')
        ux = DS.u.differentiate('x')
        uy = DS.u.differentiate('y')
        uz = DS.u.differentiate('sigma')
        vx = DS.v.differentiate('x')
        vy = DS.v.differentiate('y')
        vz = DS.v.differentiate('sigma')
        
        px = DS.p.differentiate('x')
        pxx = px.differentiate('x')
        py = DS.p.differentiate('y')
        pyy = py.differentiate('y')
        hx = DS.h.differentiate('x')
        hxx = hx.differentiate('x')
        hy = DS.h.differentiate('y')
        hyy = hy.differentiate('y')
            
        g = 9.81
        R0 = 1027
        f = gsw.f(36.5)
        div = ux+vy
        Q1 = -ux*rx*g/R0-vx*ry*g/R0
        Q2 = -uy*rx*g/R0-vy*ry*g/R0
        nonlin = -(ux**2+vy**2+2*vx*uy+wx*uz+wy*vz)
        ag2 = -g*(hxx+hyy)
        ag3 = DS.vor*f-(pxx+pyy)/R0
        ag = ag2+ag3
        vor2 = wy-vz
        vor3 = uz-wx
        
        ML = DS.zc.values
        ML[np.where(DS.rho > (DS.rho[-1,:,:]+0.03))] = np.nan
        ML = np.nanmin(ML,axis = 0)
        
        x = xr.DataArray(df1['x'].loc[d,:], dims='ID')
        y = xr.DataArray(df1['y'].loc[d,:], dims='ID')
        z = xr.DataArray(df1['z'].loc[d,:], dims='ID')
        
        df1rx.loc[d] = rx.interp(x=x, y=y, sigma = z)
        df1ry.loc[d] = ry.interp(x=x, y=y, sigma = z)
        df1rz.loc[d] = rz.interp(x=x, y=y, sigma = z)
        df1vor2.loc[d] = vor2.interp(x=x, y=y, sigma = z)
        df1vor3.loc[d] = vor3.interp(x=x, y=y, sigma = z)
        
        df1div.loc[d] = div.interp(x=x, y=y, sigma = z)
        df1Q1.loc[d] = Q1.interp(x=x, y=y, sigma = z)
        df1Q2.loc[d] = Q2.interp(x=x, y=y, sigma = z)
        df1nonlin.loc[d] = nonlin.interp(x=x, y=y, sigma = z)
        df1ag.loc[d] = ag.interp(x=x, y=y, sigma = z)
        df1wz.loc[d] = wz.interp(x=x, y=y, sigma = z)
        
        MLfunction = rgi((DS.yc*1000,DS.xc*1000),ML,bounds_error = False)
        dfML.loc[d] = MLfunction(np.array((y,x)).T)
        
        DS0.close()
        DS1.close()
        DS.close()
    dfdict['df'+str(d0)+'div'] = df1div
    dfdict['df'+str(d0)+'Q1'] = df1Q1
    dfdict['df'+str(d0)+'Q2'] = df1Q2
    dfdict['df'+str(d0)+'nonlin'] = df1nonlin
    dfdict['df'+str(d0)+'ag'] = df1ag
    dfdict['df'+str(d0)+'wz'] = df1wz
    dfdict['df'+str(d0)+'rx'] = df1rx
    dfdict['df'+str(d0)+'ry'] = df1ry
    dfdict['df'+str(d0)+'rz'] = df1rz
    dfdict['df'+str(d0)+'ml'] = df1ml
    dfdict['df'+str(d0)+'vor2'] = df1vor2
    dfdict['df'+str(d0)+'vor3'] = df1vor3
    dfdict['df'+str(d0)+'ML'] = dfML

#### Find the subduction time

In [8]:
for d0 in np.arange(44,63):
    z1 = dfdict['df'+str(d0)]['z']
    r1 = dfdict['df'+str(d0)+'rz']
    ml = dfdict['df'+str(d0)+'ML']
    subducted = (z1.iloc[-1,:] < (ml.iloc[-1,:]-5)) & (z1.iloc[0,:] > ml.iloc[0,:]) & (z1.iloc[-1,:]+5 < -5)
    downtime = (z1.loc[:,subducted] < (ml.loc[:,subducted]-5)) & (z1.loc[:,subducted] < -5)
    time1 = pd.DataFrame(index=downtime.index, columns=downtime.columns)
    for ID in downtime.columns:
        time1.loc[:,ID] = downtime.index - downtime.index[downtime.loc[:,ID]][0]
    dfdict['time'+str(d0)] = time1

#### Compute frontogenesis terms

In [4]:
## Interpolate gradients onto the particles so that we can quantify which frontogenetic terms are important
g = 9.81
R0 = 1027
f = gsw.f(36.9)
k = 1
v = 1e-5
for d0 in np.arange(44,62):
    # open particle positions
    df1 = dfdict['df'+str(d0)]
    # initialize dataframes
    df1Q1 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Q2 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Qg1 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Qg2 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Qw = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Qkh = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Qkv = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1div = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    df1Qkh1 = pd.DataFrame(index=df1.index, columns=df1['x'].columns)
    for d in df1.index.values:
        day0 = int(np.floor(d*86400/108/100)*100)
        day1 = day0+100
        dd = np.mod(int(d*86400/108),100)
        filename = './../../exp_IRENE_1wiggle_cooled_bottom_friction/full_'+str(day0)+'.cdf'
        DS0 = xr.open_dataset(filename)
        DS0 = DS0.assign_coords(x = DS0.xc*1000,y = DS0.yc*1000,sigma = DS0.zc[:,1,1])
        filename = './../../exp_IRENE_1wiggle_cooled_bottom_friction/full_'+str(day1)+'.cdf'
        DS1 = xr.open_dataset(filename)
        DS1 = DS1.assign_coords(x = DS1.xc*1000,y = DS1.yc*1000,sigma = DS0.zc[:,1,1])
        DS = DS0*(1-dd/100)+DS1*dd/100

        rx = DS.rho.differentiate('x')
        ry = DS.rho.differentiate('y')
        rz = DS.rho.differentiate('sigma')
        hx = DS.h.differentiate('x')
        hy = DS.h.differentiate('y')
        
        M4 = (rx**2+ry**2)*g**2/R0**2
        M4x = M4.differentiate('x')
        M4y = M4.differentiate('y')
        M4xx = M4x.differentiate('x')
        M4yy = M4y.differentiate('y')
             
        rxx = rx.differentiate('x')
        ryy = ry.differentiate('y')
        rzz = rz.differentiate('sigma')
             
        rxxx = rxx.differentiate('x')
        ryyx = ryy.differentiate('x')
        rzzx = rzz.differentiate('x')
             
        rxxy = rxx.differentiate('y')
        ryyy = ryy.differentiate('y')
        rzzy = rzz.differentiate('y')
             
        ## Calcuate geostrophic velocity
        ugeo = xr.DataArray(np.empty((66,322,258)),dims=['sigma','y','x'])
        vgeo = xr.DataArray(np.empty((66,322,258)),dims=['sigma','y','x'])
        ugeo[-1,:,:] = -g/f*hy
        vgeo[-1,:,:] = g/f*hx
        for k in np.arange(1,66):
            ugz = np.trapz(-ry[k:,:,:]*g/R0/f,DS.zc[k:,:,:],axis = 0)
            vgz = np.trapz(rx[k:,:,:]*g/R0/f,DS.zc[k:,:,:],axis = 0)
            ugeo[k,:,:] = ugeo[-1,:,:]+ugz
            vgeo[k,:,:] = vgeo[-1,:,:]+vgz
        
        ugeo = ugeo.assign_coords(x = DS0.xc*1000,y = DS0.yc*1000,sigma = DS0.zc[:,1,1])
        vgeo = vgeo.assign_coords(x = DS0.xc*1000,y = DS0.yc*1000,sigma = DS0.zc[:,1,1])
        ua = DS.u-ugeo
        va = DS.v-vgeo
            
        ux = ua.differentiate('x')
        uy = ua.differentiate('y')
        vx = va.differentiate('x')
        vy = va.differentiate('y')
            
        wx = DS.w.differentiate('x')
        wy = DS.w.differentiate('y')
             
        ugx = ugeo.differentiate('x')
        ugy = ugeo.differentiate('y')
        vgx = vgeo.differentiate('x')
        vgy = vgeo.differentiate('y')
            
        Q1 = -ux*rx*g/R0-vx*ry*g/R0
        Q2 = -uy*rx*g/R0-vy*ry*g/R0
        Qg1 = -ugx*rx*g/R0-vgx*ry*g/R0
        Qg2 = -ugy*rx*g/R0-vgy*ry*g/R0
            
        Qw = wx*rz*g/R0*rx*g/R0+wy*rz*g/R0*ry*g/R0
        Qkh = k*g/R0*(rxxx+ryyx)*g/R0*rx+k*g/R0*(rxxy+ryyy)*g/R0*ry
        Qkv = v*g/R0*rzzx*g/R0*rx+v*g/R0*rzzy*g/R0*ry
        Qkh1 = k*(M4xx+M4yy)
            
        div = ux+vy
            
        x = xr.DataArray(df1['x'].loc[d,:], dims='ID')
        y = xr.DataArray(df1['y'].loc[d,:], dims='ID')
        z = xr.DataArray(df1['z'].loc[d,:], dims='ID')
             
        df1Q1.loc[d] = Q1.interp(x=x, y=y, sigma = z)
        df1Q2.loc[d] = Q2.interp(x=x, y=y, sigma = z)
        df1Qg1.loc[d] = Qg1.interp(x=x, y=y, sigma = z)
        df1Qg2.loc[d] = Qg2.interp(x=x, y=y, sigma = z)
        df1Qw.loc[d] = Qw.interp(x=x, y=y, sigma = z)
        df1Qkh.loc[d] = Qkh.interp(x=x, y=y, sigma = z)
        df1Qkv.loc[d] = Qkv.interp(x=x, y=y, sigma = z)
        df1div.loc[d] = div.interp(x=x, y=y, sigma = z)
        df1Qkh1.loc[d] = Qkh1.interp(x=x, y=y, sigma = z)
     
        DS0.close()
        DS1.close()
    dfdict['df'+str(d0)+'Q1'] = df1Q1
    dfdict['df'+str(d0)+'Q2'] = df1Q2
    dfdict['df'+str(d0)+'Qg1'] = df1Qg1
    dfdict['df'+str(d0)+'Qg2'] = df1Qg2
    dfdict['df'+str(d0)+'Qw'] = df1Qw
    dfdict['df'+str(d0)+'Qkh'] = df1Qkh
    dfdict['df'+str(d0)+'Qkv'] = df1Qkv
    dfdict['df'+str(d0)+'div'] = div
    dfdict['df'+str(d0)+'Qkh1'] = df1Qkh1

#### Save particle dictionary with variables

In [94]:
f = open("particles_with_gradients.pkl","wb")
pickle.dump(dfdict,f)
f.close()